In [43]:
""" import """
import os
from dotenv import load_dotenv

import openai

from langchain import OpenAI, LLMChain, PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import ( SequentialChain, SimpleSequentialChain )
from langchain.memory import SimpleMemory

In [31]:
""" set OpenAI API key """
load_dotenv(dotenv_path="../../../.env")
openai.api_key = os.getenv("OPENAI_API_KEY")

llm = OpenAI(openai_api_key=openai.api_key, temperature=0.9)
chat = ChatOpenAI(openai_api_key=openai.api_key, temperature=0.9)

In [34]:
""" 
Sequential
make a series of calls to a language model. 
This is particularly useful when you want to take the output from one call and use it as the input to another.
"""

# This is an LLMChain to write a synopsis given a title of a play.
template = """You are a playwright. Given the title of play, it is your job to write a synopsis for that title.

Title: {title}
Playwright: This is a synopsis for the above play:"""

prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

# This is an LLMChain to write a review of a play given a synopsis.
template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

In [35]:
"""
SimpleSequentialChain
the simplest form of sequential chains, where each step has a singular input/output, and the output of one step is the input to the next.
"""
overall_chain = SimpleSequentialChain(chains=[synopsis_chain, review_chain], verbose=True)

review = overall_chain.run("Tragedy at sunset on the beach")



> Entering new SimpleSequentialChain chain...


Tragedy at Sunset on the Beach is a heart-wrenching story of love and loss set on a beach on the last day of summer. Two lifelong friends, John and Sarah, have been in love since they were children, but their relationship has been kept hidden from the world. On this fateful day, they finally decide to take the plunge and make their relationship public. Unfortunately, tragedy strikes before they can make it official, and John is taken from Sarah far too soon. The story follows Sarah as she grieves John's loss and tries to make sense of this sudden tragedy. Along the way, she comes to understand the depth of her love for John and how precious life can be. Ultimately, Tragedy at Sunset on the Beach is a bittersweet story of love and the fragility of life.


Tragedy at Sunset on the Beach is a beautiful and heartbreaking story that will have you in tears from beginning to end. John and Sarah's story of love, loss, and grief is told with emo

In [36]:
print(review) # review of self-made synopsis



Tragedy at Sunset on the Beach is a beautiful and heartbreaking story that will have you in tears from beginning to end. John and Sarah's story of love, loss, and grief is told with emotion and depth rarely seen on stage. The actors' performances capture the tragedy of the situation, allowing the audience to feel the pain and sorrow of the characters. The script is both lyrical and powerful, offering a unique take on the idea of unrequited love. 

The set design is also noteworthy, helping to create a beautiful and tranquil backdrop for this moving story. The setting of the beach really adds to the drama of the play, serving as the last hurrah for John and Sarah's relationship. The play is ultimately a reminder of the preciousness of life and love and how fragile they can be. 

Tragedy at Sunset on the Beach is a powerful and moving play that will stay with you long after you leave the theatre. It is a must-see for anyone who is a fan of romance, tragedy, and the power of love.


In [39]:
"""
SequentialChain
more complex chains that involve multiple inputs, and where there also multiple final outputs
"""

# This is an LLMChain to write a synopsis given a title of a play and the era it is set in.
llm = OpenAI(temperature=.7)
template = """You are a playwright. Given the title of play and the era it is set in, it is your job to write a synopsis for that title.

Title: {title}
Era: {era}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title", "era"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="synopsis")

# This is an LLMChain to write a review of a play given a synopsis.
llm = OpenAI(temperature=.7)
template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review")

In [42]:
# This is the overall chain where we run these two chains in sequence.
overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True)

overall_chain({"title":"Tragedy at sunset on the beach", "era": "Victorian England"})



> Entering new SequentialChain chain...

> Finished chain.


{'title': 'Tragedy at sunset on the beach',
 'era': 'Victorian England',
 'synopsis': "\n\nTragedy at Sunset on the Beach is set in Victorian England and tells the story of a young couple, Edward and Elizabeth, whose relationship is on the brink of disaster. Edward has been struggling with a gambling addiction that has left him in a desperate financial state and his behavior has become erratic and dangerous. Elizabeth is desperate to save their relationship but Edward refuses to seek help, believing that it will only make matters worse. \n\nOne day, Edward takes Elizabeth to the beach in a last-ditch effort to save their relationship. As they watch the sunset, they reminisce on happier times and Elizabeth attempts to convince Edward to seek help, but he still refuses. Suddenly, a group of men appear on the beach and confront Edward, demanding he pay his gambling debts. Edward refuses and a violent altercation ensues, leaving Edward dead on the beach. \n\nElizabeth is left devastated an

In [44]:
""" 
Memory in Sequential Chains 
pass along some context to use in each step of the chain or in a later part of the chain.
Using SimpleMemory is a convenient way to do manage this and clean up your chains
"""

template = """You are a social media manager for a theater company.  Given the title of play, the era it is set in, the date,time and location, the synopsis of the play, and the review of the play, it is your job to write a social media post for that play.

Here is some context about the time and location of the play:
Date and Time: {time}
Location: {location}

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:
{review}

Social Media Post:
"""

prompt_template = PromptTemplate(input_variables=["synopsis", "review", "time", "location"], template=template)
social_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="social_post_text")

In [45]:
overall_chain = SequentialChain(
    memory=SimpleMemory(memories={"time": "December 25th, 8pm PST", "location": "Theater in the Park"}),
    chains=[synopsis_chain, review_chain, social_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["social_post_text"],
    verbose=True)

overall_chain({"title":"Tragedy at sunset on the beach", "era": "Victorian England"})



> Entering new SequentialChain chain...

> Finished chain.


{'title': 'Tragedy at sunset on the beach',
 'era': 'Victorian England',
 'time': 'December 25th, 8pm PST',
 'location': 'Theater in the Park',
 'social_post_text': "Experience the power of forgiveness at ‘Tragedy at Sunset on the Beach’, playing December 25th at 8pm PST at Theater in the Park. This emotionally gripping production will take you on an exploration of the complexities of human relationships and the devastating consequences of unbridled ambition. Don't miss this must-see production - get your tickets now! #TragedyAtSunsetOnTheBeach #TheaterInThePark #Forgiveness"}